In [1]:
from datetime import datetime, timedelta
import pytz
import json
import requests
import pandas as pd
import time 
import numpy as np
from pandas.io.json import json_normalize
# from keboola import docker

In [2]:
with open('data.txt', 'r') as myfile:
    API_token=myfile.read().replace('\n', '')

parameters = {  'authtoken': API_token,
                'flag': 'internal'}

In [3]:
def get_data_projects(target_df, endpoint, to_append, target_id = '', url = ''):
    
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    
    if req.status_code != 200:
        print('Warning: The returned status code is: ' + str(req.status_code))
        return(target_df)
    
    else:
        df_tmp = json_normalize(req.json()[endpoint][0])
        df_tmp['parent'] = to_append
        
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)

def get_data_tasklists(target_df, endpoint, to_append, target_id = '', url = ''):
    
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    df_tmp = json_normalize(req.json()[endpoint])
    df_tmp['parent'] = to_append
    
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)
        
def get_data_tasks(target_df, endpoint, to_append, target_id = '', url = ''):
    
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    
    if req.status_code != 200:
        print('Warning: The returned status code is: ' + str(req.status_code))
        return(target_df)
    
    else:
        df_tmp = json_normalize(req.json()[endpoint])
        df_tmp['parent'] = to_append
    
    
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)

def get_data_timelogs(target_df, endpoint, to_append, target_id = '', url = ''):
    if target_id != '':
        url = url + str(target_id) + '/'
    req = requests.get(url = url, params = parameters)
    
    if req.status_code != 200:
        print('Warning: The returned status code is: ' + str(req.status_code))
        return(target_df)
    
    else:
        df_tmp = json_normalize(req.json()[endpoint]['tasklogs'])
        df_tmp['parent'] = to_append
    
    
    if target_df.empty:
        target_df = df_tmp
    else:
        target_df = target_df.append(df_tmp)
  
    return(target_df)
        


In [4]:
url = 'https://projectsapi.zoho.eu/restapi/portal/20061034739/projects/'
req = requests.get(url=url, params = parameters)
base_df = json_normalize(req.json()['projects'])

projects_info = pd.DataFrame(np.zeros((0,0)))
for i in base_df['link.self.url']:
    projects_info = get_data_projects(target_df = projects_info,
                             url = i, endpoint = 'projects',
                             to_append ='Keboola')  

In [6]:
tasklists_info = pd.DataFrame(np.zeros((0,0)))
for counter, i in enumerate(projects_info['link.tasklist.url']):
    tasklists_info = get_data_tasklists(target_df = tasklists_info, url = i,
                              endpoint = 'tasklists', to_append = projects_info['id'].iloc[counter])

In [7]:
tasks_info = pd.DataFrame(np.zeros((0,0)))
for counter, i in enumerate(tasklists_info['link.task.url']):
    tasks_info = get_data_tasks(target_df = tasks_info, url = i,
                                endpoint = 'tasks',
                                to_append = tasklists_info['id'].iloc[counter])

In [13]:
tasks_info['link.timesheet.url'].iloc[0]

'https://projectsapi.zoho.eu/restapi/portal/20061034739/projects/15944000000300003/tasks/15944000000302597/logs/'

In [14]:
timelogs_info = pd.DataFrame(np.zeros((0,0)))
for counter, i in enumerate(tasks_info['link.timesheet.url']):
    timelogs_info = get_data_timelogs(target_df = timelogs_info, url = i,
                                endpoint = 'timelogs',
                                to_append = tasks_info['id'].iloc[counter])

In [15]:
timelogs_info

,approval_status,bill_status,created_date,created_time_format,created_time_long,hours,hours_display,id,id_string,last_modified_date,...,log_date_long,minutes,notes,owner_id,owner_name,task.id,task.id_string,task.name,total_minutes,parent
0,Approved,Non Billable,08-16-2018,08-16-2018 03:32:09 PM,1534429929667,6,06:00,15944000000301731,15944000000301731,08-16-2018,...,1534460399000,0,designing the architecture\r\nadding unique id...,20064516690,radim.kasparek,15944000000302597,15944000000302597,NLP - preprocessing,360,15944000000302597
0,Approved,Non Billable,08-16-2018,08-16-2018 03:32:27 PM,1534429947902,1,01:00,15944000000302621,15944000000302621,08-16-2018,...,1534460399000,0,getting meself familiar with the tool\r\ntesti...,20064516690,radim.kasparek,15944000000302603,15944000000302603,NLP,60,15944000000302603
0,Approved,Non Billable,08-15-2018,08-15-2018 05:37:08 PM,1534351028706,8,08:00,15944000000301645,15944000000301645,08-15-2018,...,1534373999000,0,manual mapping\r\npython matching based on add...,20064516690,radim.kasparek,15944000000301097,15944000000301097,Connecting Sources,480,15944000000301097
1,Approved,Non Billable,08-14-2018,08-14-2018 06:43:13 PM,1534268593783,3,03:00,15944000000302083,15944000000302083,08-14-2018,...,1534287599000,0,unfortunately the location is not sufficient f...,20064516690,radim.kasparek,15944000000301097,15944000000301097,Connecting Sources,180,15944000000301097
0,Approved,Non Billable,08-08-2018,08-08-2018 04:31:21 PM,1533742281350,1,01:00,15944000000299087,15944000000299087,08-08-2018,...,1533769199000,0,"the Graph API is a dead end, we need to use Ap...",20064516690,radim.kasparek,15944000000299061,15944000000299061,Facebook Graph API Extractor - checking out,60,15944000000299061
1,Approved,Non Billable,08-08-2018,08-08-2018 04:31:45 PM,1533742305941,2,02:00,15944000000299091,15944000000299091,08-08-2018,...,1533682799000,0,reading through the documentation of the Graph...,20064516690,radim.kasparek,15944000000299061,15944000000299061,Facebook Graph API Extractor - checking out,120,15944000000299061
0,Approved,Non Billable,08-08-2018,08-08-2018 04:33:38 PM,1533742418206,2,02:00,15944000000300289,15944000000300289,08-08-2018,...,1533769199000,0,finding about the pricing and the APIs we need...,20064516690,radim.kasparek,15944000000300231,15944000000300231,Google Place Reviews Extractor - checking out,120,15944000000300231
1,Approved,Non Billable,08-08-2018,08-08-2018 04:32:57 PM,1533742377160,2,02:00,15944000000300285,15944000000300285,08-08-2018,...,1533682799000,0,reading through the documentation of Google AP...,20064516690,radim.kasparek,15944000000300231,15944000000300231,Google Place Reviews Extractor - checking out,120,15944000000300231
0,Approved,Non Billable,08-14-2018,08-14-2018 09:50:15 AM,1534236615235,2,02:30,15944000000301093,15944000000301093,08-14-2018,...,1534287599000,30,"Setting up billing, google drive sheet, testing",20064516690,radim.kasparek,15944000000300239,15944000000300239,Google Place Reviews Extractor - testing,150,15944000000300239
0,Approved,Non Billable,08-13-2018,08-13-2018 04:37:11 PM,1534174631160,3,03:00,15944000000302013,15944000000302013,08-13-2018,...,1534201199000,0,,20064516690,radim.kasparek,15944000000300245,15944000000300245,Trip Advisor Reviews Extractor - checking out,180,15944000000300245
